In [1]:
import pandas as pd
import datetime as dt
from datetime import time,timedelta
import numpy as np
pd.set_option('display.max_columns', 110)

In [2]:
!pip install psycopg2 --proxy="http://alcobproxy:8080" --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [3]:
!pip install pykdtree --proxy="http://alcobproxy:8080" --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [4]:
#from scipy.spatial import KDTree
from pykdtree.kdtree import KDTree
from math import *

def to_Cartesian(lat, lng):
    R = 6371 # radius of the Earth in kilometers
    lat = radians(lat)
    lng = radians(lng)
    x = R * cos(lat) * cos(lng)
    y = R * cos(lat) * sin(lng)
    z = R * sin(lat)
    return x, y, z

In [5]:
#change this file
df = pd.read_csv('EV_PERIODIC_DATA_JAN11.csv')
depot = pd.read_csv('Lat_Long_EV_Ahmedabad_corrected.csv')
#changed destiation lat lon for RTO circle (vijay nagar, Ranip) and Jaimangal (Naranpura amts bus depot)

FileNotFoundError: File b'EV_PERIODIC_DATA_JAN11.csv' does not exist

In [ ]:
df=df[['OBU_ID','VIN','EVENT_UTC', 'TIMESTAMP_LOCAL','PUBLISH_UTC','LATITUDE','LONGITUDE','IGNITION_STATUS','VEHICLE_BATTERY_POTENTIAL','VEHICLE_ODO_METER','BATTERY_STATUS','CHARGER_STATUS','RECEIVED_TIMESTAMP', 'WHEEL_BASED_VEHICLE_SPEED', 'SOC', 'BATTERY_OUT']]

#df['TIMESTAMP_LOCAL1']=df['TIMESTAMP_LOCAL'].apply(lambda x: x.split('.')[0])
df['TIMESTAMP_LOCAL1']=df['TIMESTAMP_LOCAL'].str.split('.',expand = True)[0]

df['TIMESTAMP_LOCAL1'] = pd.to_datetime(df['TIMESTAMP_LOCAL1'])

#Deleting duplicates
df = df.drop_duplicates(subset= ['OBU_ID','TIMESTAMP_LOCAL1'])

#sorting
df=df.sort_values(by=['OBU_ID','TIMESTAMP_LOCAL1'],ascending=True)
df.reset_index(inplace=True,drop=True)


df['Ignition_next']= df.groupby("OBU_ID")['IGNITION_STATUS'].shift(-1)

df['TIMESTAMP_LOCAL1_next']=df.groupby(['OBU_ID'])['TIMESTAMP_LOCAL1'].shift(-1)

#stopping period based on ignition
df['time_difference']= df['TIMESTAMP_LOCAL1_next'] - df['TIMESTAMP_LOCAL1']
df['time_difference']=df['time_difference']/timedelta(minutes=1)

df['WHEEL_BASED_VEHICLE_SPEED_prev']=df.groupby(['OBU_ID'])['WHEEL_BASED_VEHICLE_SPEED'].shift(1)

df['idle_flag']=np.where( (df['WHEEL_BASED_VEHICLE_SPEED'] == 0.0) & (df['WHEEL_BASED_VEHICLE_SPEED_prev'] > 0.0), 1, 0)

df['idle_id'] =  df.groupby(['OBU_ID'])['idle_flag'].transform(pd.Series.cumsum)

#stopping period based on vehicle wheel speed

df['idle_time'] = np.where(df['WHEEL_BASED_VEHICLE_SPEED'] == 0.0, df['time_difference'],0) ##doubt

df['total_idle_time'] =  df.groupby(['OBU_ID', 'idle_id'])['idle_time'].transform(pd.Series.cumsum)

In [ ]:
depot['x1'], depot['y1'], depot['z1'] = zip(*map(to_Cartesian, depot['Used_lat'], depot['Used_lon']))
Y = list(zip(depot['x1'], depot['y1'], depot['z1']))

df['x'], df['y'], df['z'] = zip(*map(to_Cartesian, df['LATITUDE'], df['LONGITUDE']))
X = list(zip(df['x'], df['y'], df['z']))

X= np.asarray(X)
Y= np.asarray(Y)

tree = KDTree(Y)
neighbor_dists, neighbor_indices = tree.query(X, k=1)

df.drop(columns=['x', 'y', 'z'],inplace=True)

df['dist_from_dest'] = neighbor_dists

#new trip identifier
# either first record or 
# G, H, I - whenever ignition status has moved from 0 to 1 (ignition next) and the time difference is more than 2 min or
# F - total idle time > 2 min
# D - and importantly the lat long is at most 200m from one of the destinations (201/101 routes are parked 1.5 km from RTO circle, so i have changed depot destinations)
#if  there is no record for more than 30 min then reset the trip - A
#if the vehicle has stopped near the depot and we don't get any further records, then also close the trip  - J


no_records_time_check = 30.0 # in min
ignition_off_time_check = 2.0  #in min
zero_speed_time_check = 2.0  # in min
dist_from_depot_check = 0.2  # in km  # 200 m

A = (df['time_difference'] > no_records_time_check) 

D = ( df['dist_from_dest'] < dist_from_depot_check)

F = ( df['total_idle_time'] > zero_speed_time_check )
G =  (df['IGNITION_STATUS'] == 0)
H = (df['Ignition_next'] == 1 )
I = (df['time_difference'] > ignition_off_time_check)
J = (df['time_difference'].isna())

E = (G & ((H & I ) | J))
C = (E | F)
B = (C & D)


#temp_trip_flag indicates a trip closing
df['temp_trip_flag'] = np.where( (A | B) , 1, 0)

# the trip starts from the next record onwards. so move the flag one row
df['new_trip_flag'] = df.groupby(['OBU_ID'])['temp_trip_flag'].shift(1)

df['new_trip_flag'] = np.where(df['new_trip_flag'].isna() == True, 1, df['new_trip_flag'])

#generate trip id
df['trip_id'] =  df.groupby(['OBU_ID'])['new_trip_flag'].transform(pd.Series.cumsum)

#consider only those trips that have trip closing - temp_trip_flag is 1
#this may be helpful when we run the logic every 1 hour; unfinished trips will not be considered then
df['is_trip_completed'] =  df.groupby(['OBU_ID', 'trip_id'])['temp_trip_flag'].transform(pd.Series.sum)

df = df[df['is_trip_completed'] == 1]

In [ ]:
df_trip = df.groupby(['OBU_ID', 'trip_id']).agg(['first', 'last'])['TIMESTAMP_LOCAL1']

df_temp = df[df['SOC'] != 0]
df_trip_soc = df_temp.groupby(['OBU_ID', 'trip_id']).agg(['first', 'last'])['SOC']

df_temp = df[df['VEHICLE_ODO_METER'] != 0]
df_trip_odo = df_temp.groupby(['OBU_ID', 'trip_id']).agg(['first', 'last'])['VEHICLE_ODO_METER']

df= pd.merge(df,df_trip,how='left',left_on=['OBU_ID','trip_id'], right_on=['OBU_ID','trip_id'])

df = df.rename(columns = {'first':'Trip_Start', 'last':'Trip_End'})

df= pd.merge(df,df_trip_soc,how='left',left_on=['OBU_ID','trip_id'], right_on=['OBU_ID','trip_id'])

df = df.rename(columns = {'first':'SOC_Start', 'last':'SOC_End'})

df= pd.merge(df,df_trip_odo,how='left',left_on=['OBU_ID','trip_id'], right_on=['OBU_ID','trip_id'])
df = df.rename(columns = {'first':'Odo_Start', 'last':'Odo_End'})

df['trip_duration'] = (df['Trip_End'] - df['Trip_Start'])/timedelta(minutes=1)
df['trip_km'] = (df['Odo_End'] - df['Odo_Start'])

#energy consumed
df_temp = df[df['BATTERY_OUT'] != 0]
df_trip_energy = df_temp.groupby(['OBU_ID', 'trip_id']).agg(['first', 'last'])['BATTERY_OUT']

df= pd.merge(df,df_trip_energy,how='left',left_on=['OBU_ID','trip_id'], right_on=['OBU_ID','trip_id'])
df = df.rename(columns = {'first':'Battery_out_first', 'last':'Battery_out_last'})

df['Energy_consumed'] = (df['Battery_out_last'] - df['Battery_out_first'])
df['Efficiency'] = df['Energy_consumed'] / df['trip_km']

In [ ]:
df_min_dist = df.groupby(['OBU_ID', 'trip_id']).agg(['min'])['dist_from_dest']
df= pd.merge(df,df_min_dist,how='left',left_on=['OBU_ID','trip_id'], right_on=['OBU_ID','trip_id'])
df = df.rename(columns = {'min':'trip_min_dist_from_dest'})

#consider only trips with duration more than 15 min
# remove those trips that are not close to the depots (max 5 km) and have become trips as some of them are just in the beginning of file

valid_trip_duration_check = 15.0
max_dist_from_dest_check = 5.0 # in km
valid_trip_data = df[ (df['trip_duration'] > valid_trip_duration_check) & (df['trip_min_dist_from_dest'] < max_dist_from_dest_check)]

In [ ]:
valid_trip_data.to_csv('valid_trip_test.csv', index = False)